In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
import mle.tsa as mle_tsa

In [ ]:
plt.style.use('seaborn')
plt.rcParams.update({'figure.figsize': (16,8)})
sns.set_context("talk")

In [ ]:
OUTPATH = 'data/out/'

In [ ]:
outfiles = os.listdir(OUTPATH)
outfiles

In [ ]:
test_preds = []
for file in outfiles:
    print(file)
    df = pd.read_excel(OUTPATH + file, parse_dates=['Date'], index_col='Date')
    test_preds.append(df)

p_gna95 = pd.concat(test_preds, axis=0, sort=False)

In [ ]:
p_gna95['ape_cons_GNA95'] = p_gna95[['cons_GNA95', 'p_cons_GNA95']].apply(lambda x: mle_tsa.compute_ape(y_true=x[0], y_pred=x[1]), axis=1)

In [ ]:
agg_perf = (p_gna95.groupby(['model', 'split'])[['ape_cons_GNA95']].mean()*100).reset_index().rename(columns={'ape_cons_GNA95': 'MAPE'})
agg_perf

In [ ]:
p_gna95['Date'] = p_gna95.index
split_dates = p_gna95.groupby('split')['Date'].agg(['first', 'last'])
split_dates

In [ ]:
split_dates.loc["test"]

In [ ]:
test_range = split_dates.loc["test"].dt.strftime('%Y-%m-%m')
test_start_dt = test_range.values[0]
test_end_dt = test_range.values[1]

In [ ]:
sns.barplot(data=agg_perf[agg_perf['split']=='test'], x='model', y='MAPE')
plt.title(f'Promedio Error Absoluto Porcentual en Test: desde: {test_start_dt} hasta: {test_end_dt}')
plt.ylabel('MAPE (%)')
plt.show()